# Tutorial 0 - Creating a new dataset

### Step 1: Import FANFARE source code

In [11]:
import source as sc

submodule "power_class" imported


In [14]:
data_ob = sc.PowerData(data_type='energinet',line_limit=5)

Reading in new dataset
Data type: energinet
  PriceArea             datetime  GrossCon  SolarPowerProd  WindPowerProd
0       DK2  2015-05-03T11:00:00    1425.5             132     158.422801
1       DK2  2015-05-03T12:00:00    1409.6             144     231.800000
2       DK2  2015-05-03T13:00:00    1389.1             143     349.700000
3       DK2  2015-05-03T14:00:00    1374.0             141     485.375536
4       DK2  2015-05-03T15:00:00    1361.4             126     602.600000


KeyError: "['PriceArea'] not found in axis"

In [10]:
# import requests
import pandas as pd
import io
import json

url = 'https://api.energidataservice.dk/datastore_search?resource_id=electricitybalance&limit=80000'
urlData = requests.get(url).content.decode('utf-8') # this step takes a while

NameError: name 'requests' is not defined

In [ ]:
data_dict = json.loads(urlData)['result']['records']

In [ ]:
data_df = pd.DataFrame(data_dict)[['PriceArea','HourDK','GrossCon','OffshoreWindPower','OnshoreWindPower','SolarPowerProd']]
data_df['WindPowerProd'] = data_df['OffshoreWindPower'] + data_df['OnshoreWindPower']
data_df = data_df.drop(['OffshoreWindPower','OnshoreWindPower'],axis=1)
data_df = data_df.rename(columns={'HourDK':'datetime'})

In [ ]:
data_df_DK = data_df.groupby(['datetime']).sum().drop('PriceArea')
data_df_DK_DK1 = data_df_DK.merge(data_df[data_df.PriceArea == 'DK1'].drop('PriceArea',axis=1),on='datetime',suffixes=('_DK','_DK1'))
data_df_DK_DK2 = data_df_DK.merge(data_df[data_df.PriceArea == 'DK2'].drop('PriceArea',axis=1),on='datetime',suffixes=('_DK','_DK2'))
data_df_combined = data_df_DK_DK1.merge(data_df_DK_DK2.drop(['GrossCon_DK','WindPowerProd_DK','SolarPowerProd_DK'],axis=1),on='datetime')
data_df_combined.head()